In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('mol2vec.csv')
df.replace(to_replace='#NAME?',value=np.nan,inplace=True)
df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = df.columns
df = df.astype("float", errors = "ignore")
df = np.float32(df)
df[np.isnan(df)] = 0
df[np.isinf(df)] = 0
df=pd.DataFrame(df,columns=columns)
ncol = df.shape[1]-1
x = df.iloc[:,0:ncol]
y = df['active']
corr_matrix = x.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x1 = x.drop(to_drop, axis=1)
x2 = x1.loc[:, (x1 != x1.iloc[0]).any()] 
x2 = x2.round(6)
import warnings
warnings.filterwarnings("ignore")

/tmp/ipykernel_510724/1293252792.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [3]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
0,-3.042170,-3.151222,-3.232676,7.560959,5.113686,-0.568981,-19.256578,-4.969994,16.522644,1.793195,...,12.655949,20.450172,4.086627,-10.276544,-2.190987,-1.115333,-9.388687,-15.132810,-4.057350,1.0
1,-4.727887,-4.351881,-4.658237,-1.792979,-0.499149,-8.724982,-8.886705,5.068110,-1.706909,-1.742328,...,14.233682,-0.425539,-0.464745,-10.994161,-12.039774,-12.723747,-4.175037,-11.076891,2.639213,1.0
2,-4.097823,-3.983348,-5.248144,-0.722321,-1.265023,-8.232691,-7.873246,5.745502,-0.900785,-1.008448,...,13.890897,-0.387179,0.128263,-11.509965,-10.279533,-11.952808,-3.907961,-10.316779,2.601157,1.0
3,-5.616269,-6.542370,-6.168895,-3.166161,1.509026,-11.890589,-10.365218,4.797748,-1.633804,-2.156420,...,16.483412,1.412684,-0.536551,-12.108947,-14.011013,-14.057915,-5.047316,-14.451323,1.707880,1.0
4,-4.396163,-5.507332,-5.355181,-2.798300,1.738860,-10.982598,-10.979731,4.270193,-1.071588,-2.923557,...,16.781738,-0.069494,-0.499126,-10.858052,-13.415128,-14.032432,-5.018421,-12.879906,1.733685,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,-0.245050,0.000723,-1.600506,2.734911,1.502080,0.887623,-5.741633,0.586250,4.324068,-0.374940,...,4.085058,5.202066,2.300924,-5.023169,-0.517164,-1.643741,-2.033808,-4.831588,0.884858,0.0
2020,-1.848185,-1.777678,-1.303566,0.651581,1.617647,1.635964,-2.599049,-0.967981,0.539179,1.095203,...,1.162264,3.851595,-0.020075,-1.346415,-1.949291,-0.575638,0.865116,-2.355083,-1.106393,0.0
2021,0.009402,0.458835,2.047133,-0.661602,1.160643,-0.072311,-4.937521,-2.287562,4.538611,-0.010992,...,1.278592,3.704251,0.990562,-2.375479,0.759892,-2.489954,-2.155487,-2.489589,-1.650008,0.0
2022,-3.098453,-1.974883,-2.323134,-0.688243,8.347803,0.660212,-16.231108,-3.275765,4.571203,-5.276364,...,3.041915,10.691944,3.452311,-7.664165,-4.948164,-1.304520,-8.855995,-8.327632,-5.991015,0.0


In [4]:
from sklearn.model_selection import train_test_split
x_train,x_val_test,y_train,y_val_test = train_test_split(x2,y,test_size = 0.2,random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_val_test,y_val_test,test_size = 0.5,random_state = 0)

In [5]:
train = pd.concat([x_train,y_train],axis=1)
train

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1056,-0.207807,-1.364011,-3.447015,1.720522,0.808553,-0.633580,-5.552080,-1.013066,3.521325,2.361246,...,4.438786,4.774920,0.707334,-3.950082,-3.700958,-2.281516,-2.423284,-5.188316,-1.655565,0.0
805,2.961244,-3.153878,-4.702630,7.485922,1.525465,-0.986761,-15.917911,-0.663542,8.412007,3.401014,...,12.503386,15.168564,-2.349678,-7.492398,-4.603145,-6.735564,-6.906625,-15.242688,-1.446873,0.0
1390,2.347333,-0.057599,0.652247,0.007132,0.505517,0.408751,-2.140628,1.862819,0.637824,0.975961,...,4.673762,-0.160998,-1.622867,-3.396820,-1.337172,-4.176864,-1.001107,-2.435077,-0.193673,0.0
298,1.024198,-2.839672,-1.755171,6.848028,0.516792,-0.340678,-10.685789,0.209897,8.479603,1.170375,...,10.867065,8.662798,0.617745,-9.439522,-3.128908,-5.173088,-4.248036,-9.229567,0.411368,1.0
52,2.933998,0.103875,-4.375192,6.284137,4.990062,2.396582,-14.459161,1.177130,6.137638,2.818402,...,9.047378,13.668683,-1.170549,-9.306398,-2.769418,-5.402748,-5.343800,-10.687192,-5.597492,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2.901204,-4.719590,-3.342420,7.448326,0.327245,1.488382,-10.569924,0.570997,5.719680,3.771444,...,12.654892,9.298195,-1.932950,-8.559093,-4.059663,-4.757719,-3.292013,-11.052300,-1.677100,0.0
1216,-4.100272,-1.177421,-2.989299,-1.669587,7.817950,1.244616,-13.779987,-4.166894,8.368389,-6.257930,...,6.621520,10.800359,3.675624,-8.242443,-4.191943,-0.872804,-12.253287,-12.129203,-7.617924,0.0
1653,1.480806,0.095987,-2.182062,3.154430,1.149957,1.053005,-3.588453,-0.029715,1.703170,0.182078,...,3.231322,4.180483,-1.296432,-2.927768,-1.133965,-2.159557,-0.782912,-5.327051,-1.523063,0.0
559,-1.962403,-7.753722,-5.856366,6.850247,-0.415824,-6.379946,-17.161098,1.971402,6.239536,0.560266,...,21.567972,5.423640,-1.980732,-14.834678,-9.698753,-12.238006,-6.096058,-15.759419,-0.051813,1.0


In [6]:
train.to_csv('mol2vec_train.csv')

In [7]:
val = pd.concat([x_val,y_val],axis=1)
val

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
77,3.232739,-4.475724,-4.456778,11.205893,4.238486,1.374469,-18.065187,-1.016318,9.848120,3.385553,...,12.349694,17.899656,0.259693,-11.008471,-5.842949,-7.116314,-7.459478,-14.550463,-3.518305,1.0
553,-1.307224,-8.794415,-6.762803,5.914366,-0.971748,-5.616812,-18.920013,2.227596,4.709213,1.428062,...,22.840277,6.136966,-1.189645,-17.404833,-9.374476,-12.478669,-6.264454,-17.533234,0.862982,1.0
986,1.820334,-24.521311,-15.779518,0.142012,2.726208,-21.895023,-25.262663,0.883890,-8.984430,-4.322644,...,40.529377,-0.644111,-2.135378,-30.066427,-31.714384,-30.700588,-13.535532,-39.396645,7.808868,1.0
997,0.317101,-1.191741,-4.033708,0.624602,0.618727,0.995658,-6.523489,0.062853,3.394064,2.682755,...,8.167078,10.022246,3.115436,-6.226286,-2.120966,-3.563980,-5.717959,-8.376678,-4.683996,1.0
1588,0.958031,0.274439,-1.891158,2.584714,-1.128486,1.233397,-5.342284,0.631880,2.100928,1.732952,...,4.777075,4.672312,-2.101280,-3.624428,-0.820602,-1.864372,-1.895142,-3.921964,-1.601908,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,2.798941,-3.891362,-4.916494,7.078438,5.410722,2.166087,-16.995283,0.933115,6.918398,3.466037,...,5.620762,15.137353,0.530533,-10.393365,-8.302362,-5.534828,-7.222966,-12.240122,-4.800080,1.0
302,-2.932202,-3.554746,-2.952618,-0.208799,-0.131434,-8.549384,-9.426855,4.993785,-2.247911,-0.785689,...,13.905162,-0.169033,0.076000,-12.667954,-11.314098,-11.465195,-4.539279,-10.448353,1.420576,1.0
1175,-0.722500,-1.090714,-2.338301,0.901351,3.714438,-0.152781,-9.136511,-0.759429,7.277465,-1.025926,...,6.124113,7.920097,2.345562,-6.231701,-2.501891,-2.964204,-5.601058,-9.963025,-4.145179,0.0
289,2.529210,-5.469914,-3.985345,6.909398,1.304129,-0.813306,-8.712173,-0.475816,3.843769,2.127604,...,12.257727,6.569028,-4.270763,-8.238624,-7.581604,-9.796189,-3.668342,-10.774604,-1.221559,1.0


In [8]:
val.to_csv('mol2vec_valid.csv')

In [9]:
test1 = pd.concat([x_test,y_test],axis=1)
test1

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1211,1.669299,-1.245279,-1.388021,2.808105,-3.033358,2.744201,-4.043591,1.039998,2.653325,3.809708,...,5.856113,7.278094,-1.423908,-2.765087,-0.704691,-2.340606,-1.280593,-2.927471,-1.884788,0.0
249,2.710591,-5.569710,-5.761839,13.594990,-1.778300,-0.060049,-16.523672,-0.692022,10.205357,5.287248,...,13.285303,15.615888,1.169717,-11.213464,-1.494972,-4.061957,-3.518608,-11.668514,-1.219022,1.0
1379,-1.430417,-2.836802,-2.832641,5.122944,-1.670757,1.882401,-7.141111,-1.528228,6.438561,6.381274,...,6.857546,10.448311,1.614410,-3.153835,-2.633898,-3.150676,-0.211163,-6.127218,-0.901598,0.0
519,3.306672,-4.611218,-1.895927,9.800161,2.926066,1.817580,-16.226351,-1.608227,9.020052,0.719954,...,9.958804,14.205044,-0.214412,-8.948196,-0.638594,-5.095460,-5.228894,-12.690382,-2.528982,1.0
264,0.585672,-3.955715,-7.526055,6.360748,4.759942,-1.797247,-16.369093,-1.854676,9.696412,1.966624,...,13.211173,14.209587,-1.155714,-8.759263,-6.440836,-7.994546,-5.401308,-15.350376,-2.643585,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,0.140887,-1.163278,-2.282694,6.445534,-0.751128,0.014122,-6.270543,-0.020199,5.094747,2.802862,...,6.664619,6.434700,0.122961,-3.395663,-0.939790,-2.724838,-1.956820,-4.311716,-0.534355,0.0
399,-5.661010,-4.960846,-4.852810,-2.343069,0.192883,-9.584213,-9.041336,5.567705,-2.491612,-2.031075,...,14.240607,0.128373,-0.472102,-11.409333,-12.956990,-13.635090,-3.999388,-12.058227,2.556706,1.0
1320,0.334674,0.545883,0.812828,-2.177662,2.105970,-0.214091,0.444784,1.138486,1.692847,0.536540,...,1.363122,1.360651,1.278751,-1.114317,-0.086441,-1.288660,-0.673783,-2.162386,-1.869115,0.0
1064,-0.476993,0.081459,-0.506153,1.708013,2.142298,-0.100626,-5.718302,0.854035,3.165828,0.298675,...,3.214859,6.337404,0.538533,-4.285808,-0.757848,-3.504476,-1.301316,-5.080001,-2.746478,0.0


In [11]:
test1.to_csv('mol2vec_test.csv')

In [7]:
col_name = train.columns.values.tolist()
col_name.pop()

'active'

In [8]:
from pycaret.classification import *
CM = setup(data = train, target = 'active', test_data = val, silent=True, numeric_features = col_name,
             session_id=0, fold_strategy = 'stratifiedkfold', fold = 5, fold_shuffle = True, normalize = True, normalize_method = 'minmax')

,Description,Value
0,session_id,0
1,Target,active
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1619, 300)"
5,Missing Values,False
6,Numeric Features,299
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
mlp = create_model('mlp')
tunedmlp = tune_model(mlp,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedmlp)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8765,0.9368,0.8652,0.8531,0.8592,0.7493,0.7493
1,0.8364,0.8980,0.8380,0.7987,0.8179,0.6696,0.6702
2,0.8241,0.8944,0.8099,0.7931,0.8014,0.6435,0.6436
3,0.7685,0.8745,0.8239,0.7006,0.7573,0.5388,0.5453
4,0.8638,0.9207,0.7943,0.8819,0.8358,0.7200,0.7228
Mean,0.8339,0.9049,0.8263,0.8055,0.8143,0.6642,0.6663
Std,0.0377,0.0217,0.0243,0.0621,0.0344,0.0728,0.0711


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,MLP Classifier,0.8614,0.9045,0.8969,0.8286,0.8614,0.7232,0.7255


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.598826,0.773947,0.753171,0.680827,0.485809,0.743770,0.521824,0.522702,0.516202,0.687941,...,0.298573,0.664517,0.821238,0.789764,0.618429,0.722702,0.315479,1.0,1.0,0.8971
1,0.377089,0.664244,0.669544,0.500620,0.248301,0.581478,0.503075,0.692802,0.428586,0.622036,...,0.234480,0.520225,0.753862,0.677564,0.648100,0.669601,0.506696,1.0,1.0,0.9137
2,0.529843,0.264751,0.342558,0.304037,0.416872,0.203602,0.363962,0.622343,0.195117,0.428420,...,0.192657,0.234600,0.327651,0.296296,0.467567,0.280380,0.809842,1.0,1.0,0.9120
3,0.456423,0.857367,0.768513,0.320472,0.320803,0.734977,0.774968,0.579291,0.406164,0.664279,...,0.424860,0.772395,0.892248,0.864091,0.661669,0.832610,0.264603,1.0,0.0,0.8193
4,0.487727,0.894611,0.846212,0.387225,0.241156,0.740495,0.800875,0.609128,0.384116,0.632301,...,0.194165,0.831089,0.917057,0.899653,0.756585,0.911914,0.399118,0.0,0.0,0.9322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.577639,0.788791,0.736500,0.540263,0.539245,0.762146,0.545290,0.624924,0.466251,0.690651,...,0.310550,0.678392,0.774317,0.822854,0.624301,0.763831,0.259537,1.0,1.0,0.9218
198,0.297723,0.797342,0.807718,0.292090,0.286607,0.513402,0.711288,0.837852,0.309971,0.547503,...,0.290449,0.627081,0.716857,0.698770,0.690934,0.795729,0.531031,1.0,1.0,0.9360
199,0.405648,0.859933,0.829996,0.329897,0.461920,0.708317,0.717656,0.536173,0.472373,0.539415,...,0.390815,0.772273,0.884981,0.876641,0.664572,0.804369,0.288120,0.0,0.0,0.7313
200,0.564465,0.748693,0.770267,0.534506,0.352046,0.692984,0.726963,0.551045,0.413831,0.645588,...,0.098225,0.727000,0.788068,0.733691,0.712559,0.789921,0.415718,1.0,1.0,0.8598


In [10]:
xgb = create_model('xgboost')
tunedxgb = tune_model(xgb,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedxgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9397,0.8936,0.7925,0.8400,0.7030,0.7074
1,0.8179,0.9132,0.8732,0.7515,0.8078,0.6366,0.6431
2,0.8272,0.9189,0.9296,0.7416,0.8250,0.6585,0.6750
3,0.7778,0.8877,0.8099,0.7188,0.7616,0.5549,0.5583
4,0.8576,0.9308,0.8723,0.8146,0.8425,0.7128,0.7142
Mean,0.8265,0.9181,0.8757,0.7638,0.8154,0.6532,0.6596
Std,0.0285,0.0178,0.0389,0.0349,0.0296,0.0566,0.0566


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8812,0.9261,0.9072,0.8544,0.8800,0.7626,0.7639


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.598826,0.773947,0.753171,0.680827,0.485809,0.743770,0.521824,0.522702,0.516202,0.687941,...,0.298573,0.664517,0.821238,0.789764,0.618429,0.722702,0.315479,1.0,1.0,0.9986
1,0.377089,0.664244,0.669544,0.500620,0.248301,0.581478,0.503075,0.692802,0.428586,0.622036,...,0.234480,0.520225,0.753862,0.677564,0.648100,0.669601,0.506696,1.0,1.0,0.9923
2,0.529843,0.264751,0.342558,0.304037,0.416872,0.203602,0.363962,0.622343,0.195117,0.428420,...,0.192657,0.234600,0.327651,0.296296,0.467567,0.280380,0.809842,1.0,1.0,0.9886
3,0.456423,0.857367,0.768513,0.320472,0.320803,0.734977,0.774968,0.579291,0.406164,0.664279,...,0.424860,0.772395,0.892248,0.864091,0.661669,0.832610,0.264603,1.0,0.0,0.9532
4,0.487727,0.894611,0.846212,0.387225,0.241156,0.740495,0.800875,0.609128,0.384116,0.632301,...,0.194165,0.831089,0.917057,0.899653,0.756585,0.911914,0.399118,0.0,0.0,0.9968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.577639,0.788791,0.736500,0.540263,0.539245,0.762146,0.545290,0.624924,0.466251,0.690651,...,0.310550,0.678392,0.774317,0.822854,0.624301,0.763831,0.259537,1.0,1.0,0.9955
198,0.297723,0.797342,0.807718,0.292090,0.286607,0.513402,0.711288,0.837852,0.309971,0.547503,...,0.290449,0.627081,0.716857,0.698770,0.690934,0.795729,0.531031,1.0,1.0,0.9994
199,0.405648,0.859933,0.829996,0.329897,0.461920,0.708317,0.717656,0.536173,0.472373,0.539415,...,0.390815,0.772273,0.884981,0.876641,0.664572,0.804369,0.288120,0.0,0.0,0.7861
200,0.564465,0.748693,0.770267,0.534506,0.352046,0.692984,0.726963,0.551045,0.413831,0.645588,...,0.098225,0.727000,0.788068,0.733691,0.712559,0.789921,0.415718,1.0,1.0,0.9865


In [11]:
rf = create_model('rf')
tunedrf = tune_model(rf, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedrf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8673,0.9374,0.8511,0.8451,0.8481,0.7303,0.7303
1,0.8426,0.9100,0.8380,0.8095,0.8235,0.6815,0.6819
2,0.8210,0.9172,0.8451,0.7692,0.8054,0.6403,0.6428
3,0.7840,0.8908,0.7676,0.7466,0.7569,0.5626,0.5627
4,0.8607,0.9237,0.8227,0.8529,0.8375,0.7157,0.7160
Mean,0.8351,0.9158,0.8249,0.8047,0.8143,0.6661,0.6667
Std,0.0302,0.0154,0.0302,0.0415,0.0320,0.0603,0.0601


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8564,0.9283,0.8351,0.8617,0.8482,0.7121,0.7124


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.598826,0.773947,0.753171,0.680827,0.485809,0.743770,0.521824,0.522702,0.516202,0.687941,...,0.298573,0.664517,0.821238,0.789764,0.618429,0.722702,0.315479,1.0,1.0,0.8846
1,0.377089,0.664244,0.669544,0.500620,0.248301,0.581478,0.503075,0.692802,0.428586,0.622036,...,0.234480,0.520225,0.753862,0.677564,0.648100,0.669601,0.506696,1.0,1.0,0.8778
2,0.529843,0.264751,0.342558,0.304037,0.416872,0.203602,0.363962,0.622343,0.195117,0.428420,...,0.192657,0.234600,0.327651,0.296296,0.467567,0.280380,0.809842,1.0,1.0,0.6901
3,0.456423,0.857367,0.768513,0.320472,0.320803,0.734977,0.774968,0.579291,0.406164,0.664279,...,0.424860,0.772395,0.892248,0.864091,0.661669,0.832610,0.264603,1.0,0.0,0.7218
4,0.487727,0.894611,0.846212,0.387225,0.241156,0.740495,0.800875,0.609128,0.384116,0.632301,...,0.194165,0.831089,0.917057,0.899653,0.756585,0.911914,0.399118,0.0,0.0,0.9977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.577639,0.788791,0.736500,0.540263,0.539245,0.762146,0.545290,0.624924,0.466251,0.690651,...,0.310550,0.678392,0.774317,0.822854,0.624301,0.763831,0.259537,1.0,1.0,0.8833
198,0.297723,0.797342,0.807718,0.292090,0.286607,0.513402,0.711288,0.837852,0.309971,0.547503,...,0.290449,0.627081,0.716857,0.698770,0.690934,0.795729,0.531031,1.0,1.0,0.9546
199,0.405648,0.859933,0.829996,0.329897,0.461920,0.708317,0.717656,0.536173,0.472373,0.539415,...,0.390815,0.772273,0.884981,0.876641,0.664572,0.804369,0.288120,0.0,0.0,0.8837
200,0.564465,0.748693,0.770267,0.534506,0.352046,0.692984,0.726963,0.551045,0.413831,0.645588,...,0.098225,0.727000,0.788068,0.733691,0.712559,0.789921,0.415718,1.0,1.0,0.8360


In [12]:
et = create_model('et')
tunedet = tune_model(et, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedet)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8796,0.9418,0.8582,0.8643,0.8612,0.7549,0.7550
1,0.8426,0.9136,0.8310,0.8138,0.8223,0.6811,0.6812
2,0.8364,0.9170,0.8521,0.7908,0.8203,0.6706,0.6722
3,0.7963,0.8946,0.7746,0.7639,0.7692,0.5869,0.5870
4,0.8669,0.9208,0.8298,0.8603,0.8448,0.7283,0.7287
Mean,0.8444,0.9176,0.8291,0.8186,0.8236,0.6844,0.6848
Std,0.0287,0.0151,0.0295,0.0390,0.0311,0.0576,0.0576


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8465,0.9264,0.8247,0.8511,0.8377,0.6922,0.6925


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.598826,0.773947,0.753171,0.680827,0.485809,0.743770,0.521824,0.522702,0.516202,0.687941,...,0.298573,0.664517,0.821238,0.789764,0.618429,0.722702,0.315479,1.0,1.0,0.8491
1,0.377089,0.664244,0.669544,0.500620,0.248301,0.581478,0.503075,0.692802,0.428586,0.622036,...,0.234480,0.520225,0.753862,0.677564,0.648100,0.669601,0.506696,1.0,1.0,0.8718
2,0.529843,0.264751,0.342558,0.304037,0.416872,0.203602,0.363962,0.622343,0.195117,0.428420,...,0.192657,0.234600,0.327651,0.296296,0.467567,0.280380,0.809842,1.0,1.0,0.7118
3,0.456423,0.857367,0.768513,0.320472,0.320803,0.734977,0.774968,0.579291,0.406164,0.664279,...,0.424860,0.772395,0.892248,0.864091,0.661669,0.832610,0.264603,1.0,0.0,0.7655
4,0.487727,0.894611,0.846212,0.387225,0.241156,0.740495,0.800875,0.609128,0.384116,0.632301,...,0.194165,0.831089,0.917057,0.899653,0.756585,0.911914,0.399118,0.0,0.0,0.9938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.577639,0.788791,0.736500,0.540263,0.539245,0.762146,0.545290,0.624924,0.466251,0.690651,...,0.310550,0.678392,0.774317,0.822854,0.624301,0.763831,0.259537,1.0,1.0,0.8959
198,0.297723,0.797342,0.807718,0.292090,0.286607,0.513402,0.711288,0.837852,0.309971,0.547503,...,0.290449,0.627081,0.716857,0.698770,0.690934,0.795729,0.531031,1.0,1.0,0.9315
199,0.405648,0.859933,0.829996,0.329897,0.461920,0.708317,0.717656,0.536173,0.472373,0.539415,...,0.390815,0.772273,0.884981,0.876641,0.664572,0.804369,0.288120,0.0,0.0,0.7856
200,0.564465,0.748693,0.770267,0.534506,0.352046,0.692984,0.726963,0.551045,0.413831,0.645588,...,0.098225,0.727000,0.788068,0.733691,0.712559,0.789921,0.415718,1.0,1.0,0.8193


In [13]:
catboost = create_model('catboost')
tunedcat1 = tune_model(catboost, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedcat1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8858,0.9443,0.8652,0.8714,0.8683,0.7675,0.7675
1,0.8426,0.9092,0.8169,0.8227,0.8198,0.6801,0.6801
2,0.8333,0.9144,0.8451,0.7895,0.8163,0.6641,0.6654
3,0.7840,0.8943,0.7676,0.7466,0.7569,0.5626,0.5627
4,0.8731,0.9301,0.8227,0.8788,0.8498,0.7401,0.7413
Mean,0.8437,0.9185,0.8235,0.8218,0.8222,0.6829,0.6834
Std,0.0355,0.0173,0.0328,0.0498,0.0379,0.0711,0.0712


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8366,0.9216,0.8041,0.8478,0.8254,0.6721,0.6729


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.598826,0.773947,0.753171,0.680827,0.485809,0.743770,0.521824,0.522702,0.516202,0.687941,...,0.298573,0.664517,0.821238,0.789764,0.618429,0.722702,0.315479,1.0,1.0,0.9255
1,0.377089,0.664244,0.669544,0.500620,0.248301,0.581478,0.503075,0.692802,0.428586,0.622036,...,0.234480,0.520225,0.753862,0.677564,0.648100,0.669601,0.506696,1.0,1.0,0.8741
2,0.529843,0.264751,0.342558,0.304037,0.416872,0.203602,0.363962,0.622343,0.195117,0.428420,...,0.192657,0.234600,0.327651,0.296296,0.467567,0.280380,0.809842,1.0,1.0,0.9188
3,0.456423,0.857367,0.768513,0.320472,0.320803,0.734977,0.774968,0.579291,0.406164,0.664279,...,0.424860,0.772395,0.892248,0.864091,0.661669,0.832610,0.264603,1.0,0.0,0.9209
4,0.487727,0.894611,0.846212,0.387225,0.241156,0.740495,0.800875,0.609128,0.384116,0.632301,...,0.194165,0.831089,0.917057,0.899653,0.756585,0.911914,0.399118,0.0,0.0,0.9891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.577639,0.788791,0.736500,0.540263,0.539245,0.762146,0.545290,0.624924,0.466251,0.690651,...,0.310550,0.678392,0.774317,0.822854,0.624301,0.763831,0.259537,1.0,1.0,0.9022
198,0.297723,0.797342,0.807718,0.292090,0.286607,0.513402,0.711288,0.837852,0.309971,0.547503,...,0.290449,0.627081,0.716857,0.698770,0.690934,0.795729,0.531031,1.0,1.0,0.9837
199,0.405648,0.859933,0.829996,0.329897,0.461920,0.708317,0.717656,0.536173,0.472373,0.539415,...,0.390815,0.772273,0.884981,0.876641,0.664572,0.804369,0.288120,0.0,0.0,0.9016
200,0.564465,0.748693,0.770267,0.534506,0.352046,0.692984,0.726963,0.551045,0.413831,0.645588,...,0.098225,0.727000,0.788068,0.733691,0.712559,0.789921,0.415718,1.0,1.0,0.9010


In [14]:
lightgbm = create_model('lightgbm')
tunedlightgbm = tune_model(lightgbm,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedlightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8796,0.9440,0.8440,0.8750,0.8592,0.7541,0.7545
1,0.8241,0.9168,0.8099,0.7931,0.8014,0.6435,0.6436
2,0.8302,0.9168,0.8451,0.7843,0.8136,0.6581,0.6597
3,0.7901,0.8910,0.7746,0.7534,0.7639,0.5751,0.5752
4,0.8452,0.9243,0.7801,0.8527,0.8148,0.6823,0.6843
Mean,0.8339,0.9186,0.8107,0.8117,0.8106,0.6626,0.6635
Std,0.0291,0.0170,0.0301,0.0451,0.0305,0.0580,0.0582


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8515,0.9263,0.8041,0.8764,0.8387,0.7016,0.7038


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.598826,0.773947,0.753171,0.680827,0.485809,0.743770,0.521824,0.522702,0.516202,0.687941,...,0.298573,0.664517,0.821238,0.789764,0.618429,0.722702,0.315479,1.0,1.0,0.9503
1,0.377089,0.664244,0.669544,0.500620,0.248301,0.581478,0.503075,0.692802,0.428586,0.622036,...,0.234480,0.520225,0.753862,0.677564,0.648100,0.669601,0.506696,1.0,1.0,0.9254
2,0.529843,0.264751,0.342558,0.304037,0.416872,0.203602,0.363962,0.622343,0.195117,0.428420,...,0.192657,0.234600,0.327651,0.296296,0.467567,0.280380,0.809842,1.0,1.0,0.8222
3,0.456423,0.857367,0.768513,0.320472,0.320803,0.734977,0.774968,0.579291,0.406164,0.664279,...,0.424860,0.772395,0.892248,0.864091,0.661669,0.832610,0.264603,1.0,0.0,0.9678
4,0.487727,0.894611,0.846212,0.387225,0.241156,0.740495,0.800875,0.609128,0.384116,0.632301,...,0.194165,0.831089,0.917057,0.899653,0.756585,0.911914,0.399118,0.0,0.0,0.9944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.577639,0.788791,0.736500,0.540263,0.539245,0.762146,0.545290,0.624924,0.466251,0.690651,...,0.310550,0.678392,0.774317,0.822854,0.624301,0.763831,0.259537,1.0,1.0,0.9360
198,0.297723,0.797342,0.807718,0.292090,0.286607,0.513402,0.711288,0.837852,0.309971,0.547503,...,0.290449,0.627081,0.716857,0.698770,0.690934,0.795729,0.531031,1.0,1.0,0.9836
199,0.405648,0.859933,0.829996,0.329897,0.461920,0.708317,0.717656,0.536173,0.472373,0.539415,...,0.390815,0.772273,0.884981,0.876641,0.664572,0.804369,0.288120,0.0,0.0,0.9624
200,0.564465,0.748693,0.770267,0.534506,0.352046,0.692984,0.726963,0.551045,0.413831,0.645588,...,0.098225,0.727000,0.788068,0.733691,0.712559,0.789921,0.415718,1.0,1.0,0.7744


In [29]:
from itertools import combinations
ml = [tunedrf, tunedet, tunedlightgbm, tunedcat1, tunedxgb,tunedmlp]
a2 =  list(combinations(ml, 2))
a3 =  list(combinations(ml, 3))
a4 =  list(combinations(ml, 4))
a5 =  list(combinations(ml, 5))
a6 =  list(combinations(ml, 6))
all = a2 + a3 + a4 + a5 + a6 
print(len(all))
print(len(a2),len(a3),len(a4),len(a5),len(a6))

57
15 20 15 6 1


In [16]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['Accuracy']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8796,0.9418,0.8582,0.8643,0.8612,0.7549,0.7550
1,0.8395,0.9096,0.8169,0.8169,0.8169,0.6740,0.6740
2,0.8333,0.9129,0.8521,0.7857,0.8176,0.6646,0.6665
3,0.7901,0.8921,0.7746,0.7534,0.7639,0.5751,0.5752
4,0.8545,0.9214,0.7943,0.8615,0.8266,0.7016,0.7033
Mean,0.8394,0.9156,0.8192,0.8164,0.8172,0.6741,0.6748
Std,0.0294,0.0162,0.0323,0.0430,0.0312,0.0586,0.0587


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8416,0.9231,0.8041,0.8571,0.8298,0.6819,0.6831


(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False), MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=0.0001, valid

In [30]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['AUC']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8765,0.9451,0.8652,0.8531,0.8592,0.7493,0.7493
1,0.8457,0.9152,0.8521,0.8067,0.8288,0.6885,0.6894
2,0.8302,0.9153,0.8662,0.7736,0.8173,0.6597,0.6634
3,0.7840,0.8954,0.8028,0.7308,0.7651,0.5659,0.5681
4,0.8638,0.9294,0.8227,0.8593,0.8406,0.7218,0.7223
Mean,0.8400,0.9201,0.8418,0.8047,0.8222,0.6770,0.6785
Std,0.0322,0.0165,0.0250,0.0485,0.0317,0.0632,0.0624


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8564,0.9289,0.8454,0.8542,0.8497,0.7123,0.7123


(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=10, max_features=0.4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=2.3298312178277838e-08,
                       min_impurity_split=None, min_samples_leaf=2,
                       min_samples_split=3, min_weight_fraction_leaf=0.0,
                       n_estimators=300, n_jobs=-1, oob_score=False,
                       random_state=0, verbose=0, warm_start=False), XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.0706726963347071, max_bin=256,
              max_cat_

In [32]:
blend = blend_models([tunedrf,tunedxgb])
md = predict_model(blend,data = test1)
md
y_pred = md['Label']
y_true = md['active']
t = y_pred.astype(float)
t
from sklearn.metrics import balanced_accuracy_score
accuracy = balanced_accuracy_score(y_true, t)
accuracy

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8580,0.9418,0.8794,0.8105,0.8435,0.7140,0.7160
1,0.8302,0.9116,0.8662,0.7736,0.8173,0.6597,0.6634
2,0.8272,0.9176,0.9014,0.7529,0.8205,0.6564,0.6663
3,0.7809,0.8912,0.8099,0.7233,0.7641,0.5607,0.5639
4,0.8638,0.9288,0.8652,0.8299,0.8472,0.7244,0.7249
Mean,0.8320,0.9182,0.8644,0.7780,0.8185,0.6631,0.6669
Std,0.0294,0.0170,0.0302,0.0385,0.0297,0.0581,0.0573


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9015,0.9679,0.8969,0.8969,0.8969,0.8026,0.8026


0.901283796926668